## Download Satellite Images

In [1]:
import requests
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

# Your client credentials
client_id = '26987aa3-8fdd-4bae-b8fa-c5d68e9bf63d'
client_secret = 'FXJJQ7BA5%L,%-nIfkMRr(0x+ZvoOX8.emDbVgG2'

# Create a session to get the token
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)
token = oauth.fetch_token(token_url='https://services.sentinel-hub.com/oauth/token',
                          client_secret=client_secret)

#print(f'Token: {token}')

# SentinelHub API endpoint
url = 'https://services.sentinel-hub.com/api/v1/process'

# Request body
request_body = {
    "input": {
        "bounds": {
            "bbox": [
               1360000, 5121900, 1370000, 5131900
            ],
            "properties": {
                "crs": "http://www.opengis.net/def/crs/EPSG/0/3857"
            }
        },
        "data": [
            {
                "type": "sentinel-1-grd",
                "dataFilter": {
                    "timeRange": {
                        "from": "2019-02-02T00:00:00Z",
                        "to": "2019-04-02T23:59:59Z"
                    }
                },
                "processing": {
                    "orthorectify": "true"
                }
            }
        ]
    },
    "evalscript": """
            function setup() {
              return {
                input: ["VV", "VH"],  // Update this to match the bands you have in the data
                output: { bands: 1 }
              };
            }

            function evaluatePixel(samples) {
              const VV = samples.VV;
              const VH = samples.VH;
              const ratio = VV / VH;

              return [ratio];
            }
        """,    
    "output": {
        "width": 512,
        "height": 512,
        "responses": [
            {
                "identifier": "default",
                "format": {
                    "type": "image/png"
                }
            }
        ]
    }
}

# Bearer token for authorization
headers = {
    "Authorization": f"Bearer {token['access_token']}"
}

# Make the API call to download the image
response = requests.post(url, json=request_body, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Save the image to a file
    with open('downloaded_image.png', 'wb') as file:
        file.write(response.content)
    print("Image downloaded successfully!")
else:
    print(f"Error: {response.status_code}, {response.text}")


Image downloaded successfully!
